## Simple pipeline run

We'll setup a simple pipeline and use the `rubicon`'s sklearn integration to automatically log parameters and metrics as we fit and score the model.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

rubicon = Rubicon(persistence="memory", root_dir="root")
project = rubicon.get_or_create_project("Rubicon Pipeline Example")

X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

pipe = RubiconPipeline([('scaler', StandardScaler()), ('svc', SVC())], project)
pipe

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

### Fetch the results from Rubicon

During the pipeline run, an experiment was automatically created and the corresponding parameters and metrics logged to it. We can use the `rubicon` library to pull these experiments back or view them on the dashboard.

In [ ]:
experiments = project.experiments()
print(f"{len(experiments)} experiment(s)")
experiment = experiments[0]
experiment

In [ ]:
for param in experiment.parameters():
    print(f"{param.name}: {param.value}")

In [ ]:
for metric in experiment.metrics():
    print(f"{metric.name}: {metric.value}")

In [ ]:
from rubicon.ui import Dashboard

Dashboard(persistence="memory", root_dir="root").run_server()

## Use GridSearch

`GridSearch` is commonly used to test many different parameters across an estimator or pipeline. 
Each set of parameters tried in the grid search will be logged as an individual experiment.

In [ ]:
from pprint import pprint
from time import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

rubicon = Rubicon(persistence="filesystem", root_dir="./rubicon-root")
project = rubicon.get_or_create_project("Rubicon Grid Search Example")

X, y = make_classification(random_state=0)

pipeline = RubiconPipeline([('scaler', StandardScaler()), ('svc', SVC())], project)
pipeline

In [ ]:
parameters = {
    'scaler__with_mean': (True, False),
    'svc__kernel': ('linear', 'poly')
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1)

print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()

grid_search.fit(X, y, tags=['gridsearch'])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
experiments = project.experiments()
print(f"{len(experiments)} experiments logged with {len(experiments[0].parameters())} parameters each")

In [ ]:
from rubicon.ui import Dashboard

Dashboard(persistence="filesystem", root_dir="./rubicon-root").run_server()

In [ ]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

from rubicon import Rubicon
from rubicon.sklearn import RubiconPipeline

In [ ]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# #############################################################################
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

In [ ]:
! rm -r ./rubicon-root

In [ ]:
rubicon = Rubicon(persistence="filesystem", root_dir="./rubicon-root")
project = rubicon.get_or_create_project("Rubicon Grid Search Example")

# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = RubiconPipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
], project)

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(data.data, data.target)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
grid_search.get_params()

In [ ]:
experiments = project.experiments()
print(f"{len(experiments)} experiments logged with {len(experiments[0].parameters())} parameters each")